# Hello Mojo

Mojo is designed to be a super set of Python, and so many of the functions that you are already familiar with are present in Mojo. This document describes how to write a hello world program in Mojo.

In [ ]:
from IO import print 

#| CHECK: Hello Mojo!
print("Hello Mojo!")